In [ ]:
#   Copyright 2025 UKRI-STFC

#   Licensed under the Apache License, Version 2.0 (the "License");
#   you may not use this file except in compliance with the License.
#   You may obtain a copy of the License at

#   http://www.apache.org/licenses/LICENSE-2.0

#   Unless required by applicable law or agreed to in writing, software
#   distributed under the License is distributed on an "AS IS" BASIS,
#   WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#   See the License for the specific language governing permissions and
#   limitations under the License.
#
# Authors:
# Franck Vidal (URKI-STFC)

# RXSolutionsReader Laminography Demo

## Data format: RX Solutions

The data is in the format used by devices made by [RX Solutions](https://www.rx-solutions.com/en). The projections are saved in TIFF files. They are flatfield corrected using 16-bit unsigned integers. Metadata is saved in two different files, an XML file that can be used with orbital geometries, and a CSV file that can be used with flexible geometries.

## CIL Version

This notebook was developed using CIL v25.0.0

## Dataset
The data is available from Zenodo: https://doi.org/10.5281/zenodo.??????

It is a laminography dataset of ???. 
It was acquired with the ???? platform developed by [RX Solutions](https://www.rx-solutions.com/en) for the [MATEIS Laboratory](https://mateis.insa-lyon.fr/en) of [INSA-Lyon](https://www.insa-lyon.fr/en/).

Update this filepath to where you have saved the dataset:

In [ ]:
import os
data_path = "/run/media/fpvidal/DATA/CT/2023/DTHE/JeanYvesPart-NoFilter"
data_path = "/run/media/fpvidal/DATA/CT/2023/DTHE/theiere_70mu" # Corrupted TIFF files (projections) # No geom.csv
data_path = "/run/media/fpvidal/DATA/CT/2023/DTHE/theiere_140mu" # No geom.csv
data_path = "/run/media/fpvidal/DATA/CT/2023/DTHE/theiere-Fe-10mm"
# data_path = "/run/media/fpvidal/DATA/CT/2023/DTHE/Wagon-Cu-0.4mm-22umvx"
# data_path = "/run/media/fpvidal/DATA/CT/2023/DTHE/Wire-Cu-2mm-17.54umvx"
# data_path = "/run/media/fpvidal/DATA/CT/2023/DTHE/ZrO2-Box-Cu-1mm-31.86umvx"
# data_path = "/run/media/fpvidal/DATA/CT/2023/DTHE/ZrO2-Cu-1mm-10umvx"

file_path = os.path.join(data_path, 'geom.csv')



data_path = "/DATA/CT/2025/INSA/phylix_hexagon"

file_path = os.path.join(data_path, 'unireconstruction.xml')
# file_path = os.path.join(data_path, 'geometry.csv')

In [ ]:
import numpy as np
import gc

from cil.utilities.display import show2D, show_geometry, show_system_positions
from cil.processors import TransmissionAbsorptionConverter, Slicer, CentreOfRotationCorrector
from cil.framework import ImageGeometry
from cil.plugins.astra import FBP
from cil.utilities.jupyter import islicer, link_islicer
from cil.io.TIFF import TIFFWriter

from readers.RXSolutionsDataReader import RXSolutionsDataReader

# Loading Geometry

In [ ]:
scaling_factor = 8
number_of_slices_to_reconstruct = 500 # Use 0 to compute it automatically

if scaling_factor == 1:
    roi = None
else:
    roi = {"axis_1": [None, None, scaling_factor], "axis_2": [None, None, scaling_factor]}

In [ ]:
reader = RXSolutionsDataReader(file_path, pixel_pitch_in_mm=(0.15,0.15), mode="bin", roi=roi)

In [ ]:
acq_geom = reader.get_geometry()

In [ ]:
if acq_geom.geom_type != "CONE_FLEX":
    show_geometry(acq_geom)
else:
    show_system_positions(acq_geom)

In [ ]:
print(acq_geom)

# Loading Projections

In [ ]:
acq_data = reader.read()

In [ ]:
show2D(acq_data, origin='upper-left');

# Pre-processing

In [ ]:
data_exp = TransmissionAbsorptionConverter()(acq_data)

In [ ]:
if acq_geom.geom_type != "CONE_FLEX":
    processor = CentreOfRotationCorrector.image_sharpness("centre", "tigre")
    processor.set_input(data_exp)
    data_corr = processor.get_output()
else:
    data_corr = data_exp

# Prepare the data for Astra
data_corr.reorder(order='astra')

In [ ]:
if acq_geom.geom_type != "CONE_FLEX":
    image_geometry = data_corr.geometry.get_ImageGeometry()
        
    image_geometry.voxel_size_x = min(image_geometry.voxel_size_x, image_geometry.voxel_size_y, image_geometry.voxel_size_z)
    image_geometry.voxel_size_y = image_geometry.voxel_size_x
    image_geometry.voxel_size_z = image_geometry.voxel_size_x
else:
    # Use the system magnification to compute the voxel size
    mag = data_corr.geometry.magnification
    mean_mag = np.mean(mag)
    print("Mean magnification: ", mean_mag)
    
    voxel_size_xy = data_corr.geometry.config.panel.pixel_size[0] / mean_mag
    voxel_size_z = data_corr.geometry.config.panel.pixel_size[1] / mean_mag
    
    # Create an image geometry
    num_voxel_xy = int(np.ceil(data_corr.geometry.config.panel.num_pixels[0]))
    num_voxel_z = int(np.ceil(data_corr.geometry.config.panel.num_pixels[1]))
    
    image_geometry = ImageGeometry(num_voxel_xy, num_voxel_xy, num_voxel_z, voxel_size_xy, voxel_size_xy, voxel_size_z)

if number_of_slices_to_reconstruct > 0:
    image_geometry.voxel_num_z = number_of_slices_to_reconstruct // scaling_factor

print(image_geometry)

# Using a FDK for the reconstruction

In [ ]:
# Reconstruct using FDK
# Instantiate the reconsruction algorithm
fdk = FBP(image_geometry, data_corr.geometry)
fdk.set_input(data_corr)

# Perform the actual CT reconstruction
FDK_recon = fdk.get_output()

## Release memory

In [ ]:
del data_exp
del acq_data
del reader

gc.collect();

## Save the reconstruction as a stack of TIFF files

In [ ]:
writer = TIFFWriter(FDK_recon, os.path.join(data_path, "FDK-recon/slice"))
writer.write()

## Visualise the reconstruction

In [ ]:
islicer(FDK_recon)

# Using TV regularised least squares solved with FISTA for the reconstruction

In [ ]:
from cil.plugins.astra import ProjectionOperator
from cil.optimisation.functions import LeastSquares
from cil.plugins.ccpi_regularisation.functions import FGP_TV
from cil.optimisation.algorithms import FISTA

projector = ProjectionOperator(image_geometry, data_corr.geometry)
LS = LeastSquares(A=projector, b=data_corr)

alpha = 0.05
TV = FGP_TV(alpha=alpha, nonnegativity=True, device='gpu')
fista_TV = FISTA(initial=FDK_recon, f=LS, g=TV, update_objective_interval=10)

In [ ]:
fista_TV.run(30)
TV_recon = fista_TV.solution

## Save the reconstruction as a stack of TIFF files

In [ ]:
writer = TIFFWriter(TV_recon, os.path.join(data_path, "TV-recon/slice"))
writer.write()

## Visualise the reconstruction

In [ ]:
islicer(TV_recon)

# Compare the two reconstructions

In [ ]:
half_number_of_slices = FDK_recon.shape[0] // 2
fix_range = (min(FDK_recon.min(), TV_recon.min()), max(FDK_recon.max(), TV_recon.max()))

show2D([FDK_recon, TV_recon], origin='upper-left', fix_range=fix_range, slice_list=(('vertical',half_number_of_slices // 2)))
show2D([FDK_recon, TV_recon], origin='upper-left', fix_range=fix_range, slice_list=(('vertical',half_number_of_slices)))
show2D([FDK_recon, TV_recon], origin='upper-left', fix_range=fix_range, slice_list=(('vertical',half_number_of_slices + half_number_of_slices // 2)))

In [ ]:
link_islicer(islicer(FDK_recon), islicer(TV_recon))